In [1]:
using Random, LinearAlgebra, TensorToolbox, Combinatorics, TensorOperations

In [167]:
function tensorDiagonal(L::Vector, d::Int)
    r = length(L)
    D = zeros(tuple(repeat([r], d)...))
    for i = 1:r 
        D[tuple(repeat([i], d)...)...] = L[i]
    end;
    return D
end;

function makeRankedTensor(L::Vector, A::Matrix, d::Int)
    T = tensorDiagonal(L, d)
    for i=1:d 
        T = ttm(T, A, i)
    end;
    return T
end;

function randomTensor(n::Int, d::Int; real::Bool=false)
    T_ = randn(tuple(repeat([n], d)...)...)
    if !real
        Q = im*randn(tuple(repeat([n], d)...)...)
        T_ = T_+Q
    end;
    T = copy(T_)
    perms = permutations(1:d)
    for perm in perms
        if perm != 1:d 
            T = T + permutedims(T_, perm)
        end;
    end;
    return T/factorial(d)
end;

function complexGaussian(m, n)
    A = randn(m, n)
    B = randn(m, n)
    return A + im*B
end;

function randomRankedTensor(n, d, r; real=false)
    if !real
        A = complexGaussian(n, r)
    else 
        A = randn(n, r)
    end;
    L = ones(r)
    return makeRankedTensor(L, A, d), A
end;

function contract(T::Array, V::Array)
    d1 = length(size(T))
    d2 = length(size(V))
    return ncon((T, V), (vcat(collect(1:d2), -collect(1:d1-d2)), collect(1:d2)))
end;

function jennrich(T::Array; tol=1e-10)
    # draw random vectors or matrices for contraction
    B = randn(repeat([n], Int(iseven(d))+1)...);
    C = randn(repeat([n], Int(iseven(d))+1)...);

    # contract to tensor with largest even order strictly less than the order of d
    # d odd -> order n^{d-1} tensor
    # d even -> order n^{d-2} tensor 
    T1 = contract(T, B);
    T2 = contract(T, C);

    delt = Int(floor((d-1)/2))

    # flatten to matrices
    T1flat = reshape(T1, (n^delt, n^delt));
    T2flat = reshape(T2, (n^delt, n^delt));

    # obtain the factors
    # T real -> true factors are obtained, possibly up to sign
    # T complex -> true factors are obtained, but up to a complex scalar factor of norm 1 
    w, Ahat = eigen(T1flat*pinv(T2flat));
    Ahat = Ahat[:, abs.(w).>tol];
end;